In [64]:
import pandas as pd

In [65]:
features_path = '../data/splitted_dataset/train_0_features_.csv'
target_path = '../data/splitted_dataset/train_0_target_.csv'

In [66]:
features = pd.read_csv(features_path)
target = pd.read_csv(target_path)

In [67]:
joined_data = features.merge(target, on='id', how='inner')

# Create Dataset

In [68]:
spanish_data = joined_data[joined_data.language == 'spanish']

In [114]:
mini_data = spanish_data.sample(50000)

In [115]:
mini_data = mini_data[['title', 'category']]

In [116]:
len(mini_data)

50000

In [117]:
titles = mini_data.title.tolist()
categories = mini_data.category.tolist()

### Me quedo con las categorias que aparezcan más de 10 veces

In [111]:
[(k, v) for k, v in mini_data.category.value_counts().items() if v >= 10][:]

[('VIDEO_GAME_PREPAID_CARDS', 12),
 ('FLASHLIGHTS', 12),
 ('VEHICLE_BRAKE_DISCS', 12),
 ('KEYBOARD_AND_MOUSE_KITS', 12),
 ('VEHICLE_STICKERS', 12),
 ('MICROPHONES', 12),
 ('THERMOSES', 12),
 ('AUTOMOTIVE_BUMPER_GRILLES', 12),
 ('RANGES', 12),
 ('SWEATERS_AND_CARDIGANS', 12),
 ('DECORATIVE_BOXES', 11),
 ('SCHOOL_AND_OFFICE_PAPERS', 11),
 ('ALARMS_AND_SENSORS', 11),
 ('TREADMILLS', 11),
 ('REFRIGERATORS', 11),
 ('WALLETS', 11),
 ('MIXERS', 11),
 ('GRAPHICS_CARDS', 11),
 ('ANTIQUE_CHAIRS', 11),
 ('DESKTOP_COMPUTER_POWER_SUPPLIES', 11),
 ('SUSPENSION_CONTROL_ARM_BUSHINGS', 11),
 ('ROLLER_SKATES', 11),
 ('DOOR_AND_WINDOW_LOCKS', 11),
 ('MANGA', 11),
 ('KITCHEN_MOLDS', 11),
 ('MOTORCYCLE_FAIRINGS', 10),
 ('AUTOMOTIVE_DOORS', 10),
 ('NOTEBOOKS_AND_WRITING_PADS', 10),
 ('TABLECLOTHS', 10),
 ('DECORATIVE_BASKETS', 10),
 ('SANDER_MACHINES', 10),
 ('COLLECTIBLE_POSTCARDS', 10),
 ('VEHICLE_SPEAKERS', 10),
 ('AUTOMOTIVE_THROTTLE_BODIES', 10),
 ('TV_SMPS', 10),
 ('FOLDERS_AND_EXPANDING_FILES', 10),


In [110]:
len(mini_data.category.unique())

1219

# Make model

In [73]:
import tensorflow as tf
import tensorflow_hub as hub

In [74]:
module_url_span = "https://tfhub.dev/google/tf2-preview/nnlm-es-dim50-with-normalization/1"
module_url_port = "https://tfhub.dev/google/wiki40b-lm-pt/1"

In [75]:
embed = hub.KerasLayer(module_url_span)

In [79]:
embed(['hola lisa']).shape

TensorShape([1, 50])

In [81]:
from sklearn.pipeline import make_pipeline
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression  # use any classifier

In [80]:
class Transformer:
    def __init__(self, model):
        self.model = model

    def transform(self, x):
        return self.model(x)

    def fit(self, x, y):
        return self

In [118]:
model = make_pipeline(    
    Transformer(embed), # preprocessing
    LogisticRegression(), # model
    # postprocessing
    verbose=True
)

In [ ]:
model.fit(titles, categories)

[Pipeline] ....... (step 1 of 2) Processing transformer, total=   0.5s


/usr/local/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
/usr/local/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:469: FutureWarning: Default multi_class will be changed to 'auto' in 0.22. Specify the multi_class option to silence this warning.
  "this warning.", FutureWarning)


In [122]:
validation_set = spanish_data[spanish_data.category == 'MOVIES'].sample(10).title

In [123]:
for title in validation_set.tolist():
    print(title, model.predict([title]))

Boy & The Beast The Boy And The Beast Blu-ray Us Import ['VIDEO_GAMES']
Hostel - Part Ii - Blu-ray - Hd - Impecable  ['MOVIES']
Bailarina En La Oscuridad( Dancer In In The Dark) Dvd ['MUSIC_ALBUMS']
Ronaldo, El Fenomeno - Dvd Nuevo Original Cerrado Zona 4 ['MOVIES']
211 (2018) 211 Blu-ray Us Import ['CD_AND_DVD_PLAYERS']
Dvd :   - The Great Barrier Reef (eco Amaray Case) ['MOVIES']
Pelicula Super 8 8mm Rey Juan Carlos España Rtve ['COMICS']
Películas Harry Potter Dvd Originales ['MOVIES']
Leccion Mortal Helen Mirren Katie Holmes Dvd Import Z 2 Nvo ['DOLLS']
Mi Cielo Azul Vhs Steve Martin Rick Moranis My Blue Heaven ['MUSIC_ALBUMS']
